#Research Text part

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##preprocessing

In [2]:
!tar -xvf "/content/drive/MyDrive/IR_Project/epic_qa_cord_2020-10-22-split-corrected.tar.gz" -C "/content"  

Streaming output truncated to the last 5000 lines.
2020-10-22-new/w5yj1920.json
2020-10-22-new/dv0ulpp3.json
2020-10-22-new/8vot105e.json
2020-10-22-new/f2lja8p6.json
2020-10-22-new/46qc6skb.json
2020-10-22-new/xwly4fmn.json
2020-10-22-new/xz92zudb.json
2020-10-22-new/sglji3d8.json
2020-10-22-new/q4wa9twx.json
2020-10-22-new/osjwrlhy.json
2020-10-22-new/a5to8ems.json
2020-10-22-new/ahymosul.json
2020-10-22-new/sjgnj5s6.json
2020-10-22-new/8ikut006.json
2020-10-22-new/u3ylpzxe.json
2020-10-22-new/m72a3b8y.json
2020-10-22-new/dawba1a1.json
2020-10-22-new/syw9cwee.json
2020-10-22-new/l7bk78u8.json
2020-10-22-new/qr938cz7.json
2020-10-22-new/sg7yvsw9.json
2020-10-22-new/k2skui3u.json
2020-10-22-new/78dcls8r.json
2020-10-22-new/49zpfwtb.json
2020-10-22-new/fm6jmyxu.json
2020-10-22-new/6jtwti2w.json
2020-10-22-new/tupom7fb.json
2020-10-22-new/dgu8kxxy.json
2020-10-22-new/sx2mmzww.json
2020-10-22-new/fp75ip5f.json
2020-10-22-new/5fvmy15b.json
2020-10-22-new/ewqq5nlo.json
2020-10-22-new/wlff3b

In [3]:
import math
import os.path
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import json

json_path = '/content/2020-10-22-new/q4qkxp5u.json'
with open(json_path, 'r') as f:
    json_string = f.read()
json_data = json.loads(json_string)


In [5]:
import os
len(os.listdir("/content/2020-10-22-new"))

236035

In [6]:
import os
import json

directory = '/content/2020-10-22-new'
json_files = [f for f in os.listdir(directory)[0:30000] if f.endswith('.json')]
titles=[]
doc_id=[]
text=[]
data_each=[]
for json_file in json_files:
    json_path = os.path.join(directory, json_file)
    with open(json_path, 'r') as f:
        json_string = f.read()
    json_data = json.loads(json_string)
    titles.append(json_data['metadata']['title'])
    doc_id.append(json_data['document_id'])
    text.append(json_data['contexts'][0]['text'])


In [7]:
import os
import json

directory = '/content/2020-10-22-new'
json_files = [f for f in os.listdir(directory)[0:30000] if f.endswith('.json')]
titles=[]
doc_id=[]
text=[]
dic={}
data_each=[]
for json_file in json_files:
    json_path = os.path.join(directory, json_file)
    with open(json_path, 'r') as f:
        json_string = f.read()
    json_data = json.loads(json_string)
    dic[json_data['document_id']]=json_data['metadata']['title']


In [8]:
import json
lst_ques=[]
lst_queries=[]
json_path = '/content/drive/MyDrive/IR_Project/expert_questions_primary.json'
with open(json_path, 'r') as f:
    json_string = f.read()
json_data = json.loads(json_string)
for question in json_data:
  lst_ques.append(question['question'])
  lst_queries.append(question['query'])


##Tfidf

####get top 10 documents and top 10 answer of each document related to a query

In [9]:
# Sample queries and documents
queries = lst_ques
documents = dic
final=[]
# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Transform the documents and calculate the cosine similarity between queries and documents
tfidf_matrix = vectorizer.fit_transform(documents.values())
cosine_similarities = []
for query in queries:
    query_tfidf = vectorizer.transform([query])
    cosine_similarities.append(query_tfidf.dot(tfidf_matrix.T).toarray()[0])

# Get the top 10 document IDs for each query
num_top_documents = 10
for i, query in enumerate(queries):
    documentlst=[]
    print(f"\nTop {num_top_documents} document IDs for query '{query}':")
    top_documents = cosine_similarities[i].argsort()[-num_top_documents:][::-1]
    for j, index in enumerate(top_documents):
        doc_id = list(documents.keys())[index]
        similarity = cosine_similarities[i][index]
        print(f"{j+1}. {doc_id} (similarity: {similarity:.3f})")
        documentlst.append(doc_id)
    final.append(documentlst)          



Top 10 document IDs for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
1. 5zq5s65x (similarity: 0.417)
2. ranllo1l (similarity: 0.353)
3. yv3av15i (similarity: 0.346)
4. xtqvtleb (similarity: 0.329)
5. f5s0ntps (similarity: 0.290)
6. 99xju8ik (similarity: 0.285)
7. ayjqvncc (similarity: 0.285)
8. 4n5ufnyp (similarity: 0.274)
9. 7q6xi2xx (similarity: 0.272)
10. 3ekjqihf (similarity: 0.266)

Top 10 document IDs for query 'How do cytokine pathways link sleep and immunity to infection and COVID-19?':
1. wpdgqhx9 (similarity: 0.450)
2. pfp1a7vd (similarity: 0.302)
3. rvrgcugn (similarity: 0.302)
4. xkv6j56h (similarity: 0.267)
5. 1ogglaa3 (similarity: 0.256)
6. txyqfpif (similarity: 0.255)
7. blq9zu5z (similarity: 0.248)
8. qx6hhotk (similarity: 0.245)
9. gcbmfjty (similarity: 0.243)
10. pe1z6fx3 (similarity: 0.240)

Top 10 document IDs for query 'When is it safe to discharge a COVID-19 patient with pneumonia? ':
1. obu70kti (similarity: 0.504)
2. 6dgbv2tj (simil

In [10]:
final_sentences=[]
for i in final:
  lst_sentences=[]
  for j in i:
    with open('/content/2020-10-22-new/'+ j + '.json') as f:
      data = json.load(f)
      text = ' '.join([c['text'] for c in data['contexts']])
      sentences = text.split(".")
      lst_sentences.append(sentences)
  final_sentences.append(lst_sentences)   


In [11]:
# Loop through each query and its corresponding top documents
for i, (query, documents) in enumerate(zip(queries, final_sentences)):
    print(f"\nTop 10 sentences for query '{query}':")
    # Loop through each document and its corresponding sentences
    for j, (doc_id, sentences) in enumerate(zip(final[i], documents)):
        print(f"Document: {doc_id}")
        # Create a TfidfVectorizer object for the sentences
        sentence_vectorizer = TfidfVectorizer()
        # Transform the sentences
        sentence_tfidf_matrix = sentence_vectorizer.fit_transform(sentences)
        # Calculate the cosine similarity between the query and the sentences
        query_tfidf = sentence_vectorizer.transform([query])
        cosine_similarities = query_tfidf.dot(sentence_tfidf_matrix.T).toarray()[0]
        # Get the top 10 sentences
        top_sentences = cosine_similarities.argsort()[-10:][::-1]
        # Print the top 10 sentences
        for k, index in enumerate(top_sentences):
            sentence = sentences[index]
            print(f"   {k+1}. Sentence: {sentence.strip()}")
        print("")



Top 10 sentences for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
Document: 5zq5s65x
   1. Sentence: We read with considerable interest the article "Difference of coagulation features between severe pneumonia induced by SARS-CoV2 and non-SARS-CoV2" wrote by Yin et al
   2. Sentence: They found that patients with pneumonia SARS-CoV2 had higher platelet count than those induced by non-SARS-CoV2
   3. Sentence: Patients with severe pneumonia induced by SARS-CoV2 presented high D-dimer and fibrin degradation product (FDP) levels [3] , which are indices of active blood clot activation
   4. Sentence: In this retrospective study, authors compared coagulation parameters of patients with severe pneumonia induced by SARS-CoV2 and patients with pneumonia induced by other pathogens
   5. Sentence: In conclusion, in critically ill patients with SARS-CoV-2 pneumonia in which a "cytokine storm" joins to Virchow triad, it seems appropriate to highlight the need for use o

####Get answer to the given queries

In [12]:
# Loop through each query and its corresponding top documents
for i, (query, documents) in enumerate(zip(queries, final_sentences)):
    print(f"\nTop 5 answers for query '{query}':")
    # Loop through each document and its corresponding sentences
    for j, (doc_id, sentences) in enumerate(zip(final[i], documents)):
        print(f"Document: {doc_id}")
        # Create a TfidfVectorizer object for the sentences
        sentence_vectorizer = TfidfVectorizer()
        # Transform the sentences
        sentence_tfidf_matrix = sentence_vectorizer.fit_transform(sentences)
        # Calculate the cosine similarity between the query and the sentences
        query_tfidf = sentence_vectorizer.transform([query])
        cosine_similarities = query_tfidf.dot(sentence_tfidf_matrix.T).toarray()[0]
        # Get the top 5 sentences
        top_sentences = cosine_similarities.argsort()[-5:][::-1]
        # Create a list of the top 5 sentences
        top_sentences_list = [sentences[index] for index in top_sentences]
        # Merge the top 5 sentences into a paragraph
        paragraph = ". ".join(top_sentences_list)
        print(paragraph)
        print("")



Top 5 answers for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
Document: 5zq5s65x
We read with considerable interest the article "Difference of coagulation features between severe pneumonia induced by SARS-CoV2 and non-SARS-CoV2" wrote by Yin et al.  They found that patients with pneumonia SARS-CoV2 had higher platelet count than those induced by non-SARS-CoV2.  Patients with severe pneumonia induced by SARS-CoV2 presented high D-dimer and fibrin degradation product (FDP) levels [3] , which are indices of active blood clot activation.  In this retrospective study, authors compared coagulation parameters of patients with severe pneumonia induced by SARS-CoV2 and patients with pneumonia induced by other pathogens.  In conclusion, in critically ill patients with SARS-CoV-2 pneumonia in which a "cytokine storm" joins to Virchow triad, it seems appropriate to highlight the need for use of early anticoagulant prophylaxis in all patients

Document: ranllo1l
 The 

####Get Answer to a query given by the user

In [13]:
def get_answer(question):
    answers = []
    for i, (query, documents) in enumerate(zip(queries, final_sentences)):
        if query.lower() in question.lower():
            for j, (doc_id, sentences) in enumerate(zip(final[i], documents)):
                # Create a TfidfVectorizer object for the sentences
                sentence_vectorizer = TfidfVectorizer()
                sentence_tfidf_matrix = sentence_vectorizer.fit_transform(sentences)
                # Calculate the cosine similarity between the query and the sentences
                query_tfidf = sentence_vectorizer.transform([query])
                cosine_similarities = query_tfidf.dot(sentence_tfidf_matrix.T).toarray()[0]
                top_sentences = cosine_similarities.argsort()[-5:][::-1]
                top_sentences_list = [sentences[index] for index in top_sentences]
                # Merge the top 5 sentences into a paragraph
                paragraph = " ".join(top_sentences_list)
                answers.append(paragraph)
    # Check if any answers were found
    if len(answers) == 0:
        return "Sorry, I couldn't find an answer to your question."
    else:
        return answers[0]




In [14]:
# Test the function
question = lst_ques[0]
print(question)
answer = get_answer(question)
answer

What features of SARS-CoV2 are targeted in vaccine development? 


'We read with considerable interest the article "Difference of coagulation features between severe pneumonia induced by SARS-CoV2 and non-SARS-CoV2" wrote by Yin et al  They found that patients with pneumonia SARS-CoV2 had higher platelet count than those induced by non-SARS-CoV2  Patients with severe pneumonia induced by SARS-CoV2 presented high D-dimer and fibrin degradation product (FDP) levels [3] , which are indices of active blood clot activation  In this retrospective study, authors compared coagulation parameters of patients with severe pneumonia induced by SARS-CoV2 and patients with pneumonia induced by other pathogens  In conclusion, in critically ill patients with SARS-CoV-2 pneumonia in which a "cytokine storm" joins to Virchow triad, it seems appropriate to highlight the need for use of early anticoagulant prophylaxis in all patients'

##Universal Sentence Encoder

In [15]:
import tensorflow_hub as hub
import numpy as np

# Sample queries and documents
queries = lst_ques
documents = dic
final=[]

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Embed the documents using the Universal Sentence Encoder
document_embeddings = embed(list(documents.values()))

# Calculate the cosine similarity between queries and documents
cosine_similarities = []
for query in queries:
    # Embed the query using the Universal Sentence Encoder
    query_embedding = embed([query])[0]
    # Calculate the cosine similarity between the query and the documents
    similarities = np.inner(query_embedding, document_embeddings)
    cosine_similarities.append(similarities)

# Get the top 10 document IDs for each query
num_top_documents = 10
for i, query in enumerate(queries):
    documentlst=[]
    print(f"\nTop {num_top_documents} document IDs for query '{query}':")
    top_documents = cosine_similarities[i].argsort()[-num_top_documents:][::-1]
    for j, index in enumerate(top_documents):
        doc_id = list(documents.keys())[index]
        similarity = cosine_similarities[i][index]
        print(f"{j+1}. {doc_id} (similarity: {similarity:.3f})")
        documentlst.append(doc_id)
    final.append(documentlst)



Top 10 document IDs for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
1. rpojmv49 (similarity: 0.601)
2. wh9vvgv2 (similarity: 0.594)
3. q57gamxc (similarity: 0.584)
4. f5s0ntps (similarity: 0.578)
5. ranllo1l (similarity: 0.577)
6. 7q6xi2xx (similarity: 0.572)
7. xtqvtleb (similarity: 0.558)
8. t890r7xp (similarity: 0.545)
9. ogl05po0 (similarity: 0.539)
10. hcm21kug (similarity: 0.534)

Top 10 document IDs for query 'How do cytokine pathways link sleep and immunity to infection and COVID-19?':
1. gcbmfjty (similarity: 0.547)
2. wpdgqhx9 (similarity: 0.524)
3. 5gbhobw8 (similarity: 0.514)
4. eo48wugz (similarity: 0.513)
5. k50qvr4w (similarity: 0.509)
6. s5fz81vp (similarity: 0.497)
7. ahxm2fdg (similarity: 0.488)
8. z23oo877 (similarity: 0.467)
9. jsyngwut (similarity: 0.466)
10. x1ywhsug (similarity: 0.461)

Top 10 document IDs for query 'When is it safe to discharge a COVID-19 patient with pneumonia? ':
1. mp3qb33p (similarity: 0.579)
2. e44535dz (simil

In [16]:
final_sentences=[]
for i in final:
  lst_sentences=[]
  for j in i:
    with open('/content/2020-10-22-new/'+ j + '.json') as f:
      data = json.load(f)
      text = ' '.join([c['text'] for c in data['contexts']])
      sentences = text.split(".")
      lst_sentences.append(sentences)
  final_sentences.append(lst_sentences)   


In [17]:
import tensorflow_hub as hub
import numpy as np

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

for i, (query, documents) in enumerate(zip(queries, final_sentences)):
    print(f"\nTop 10 sentences for query '{query}':")
    # Loop through each document and its corresponding sentences
    for j, (doc_id, sentences) in enumerate(zip(final[i], documents)):
        print(f"Document: {doc_id}")
        # Embed the query and sentences using the Universal Sentence Encoder
        query_embedding = embed([query])[0]
        sentence_embeddings = embed(sentences)
        # Calculate the cosine similarity between the query and the sentences
        cosine_similarities = np.inner(query_embedding, sentence_embeddings)
        # Get the top 10 sentences
        top_sentences = cosine_similarities.argsort()[-10:][::-1]
        # Print the top 10 sentences
        for k, index in enumerate(top_sentences):
            sentence = sentences[index]
            print(f"   {k+1}. Sentence: {sentence.strip()}")
        print("")



Top 10 sentences for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
Document: rpojmv49
   1. Sentence: Thus evolutionarily conserved proteins have been the best candidature for vaccine development
   2. Sentence: Although DNA vaccine has also been attempted using RDT, but the high rate of mutation associated with COVID-19 have made such vaccines ineffective even before use
   3. Sentence: mRNA based vaccines (mRNA-1273, BNT162) also targets these spike proteins
   4. Sentence: It could be used to develop a predictable model for such pathogenic infections, preparing ourselves to take preventive action against its reoccurrence
   5. Sentence: Proteome analysis has also revealed the critical mutated proteins that are responsible for COVID-19 pathogenesis and virulence
   6. Sentence: These proteins can also act as important drug targets in combating COVID-19 infection
   7. Sentence: Its origin is debatable with both natural origin and conspiracy theory providi

####Get answer to the given queries

In [18]:
# Loop through each query and its corresponding top documents
for i, (query, documents) in enumerate(zip(queries, final_sentences)):
    print(f"\nTop 5 answers for query '{query}':")
    # Loop through each document and its corresponding sentences
    for j, (doc_id, sentences) in enumerate(zip(final[i], documents)):
        print(f"Document: {doc_id}")
        # Create a TfidfVectorizer object for the sentences
        sentence_vectorizer = TfidfVectorizer()
        # Transform the sentences
        sentence_tfidf_matrix = sentence_vectorizer.fit_transform(sentences)
        # Calculate the cosine similarity between the query and the sentences
        query_tfidf = sentence_vectorizer.transform([query])
        cosine_similarities = query_tfidf.dot(sentence_tfidf_matrix.T).toarray()[0]
        # Get the top 5 sentences
        top_sentences = cosine_similarities.argsort()[-5:][::-1]
        # Create a list of the top 5 sentences
        top_sentences_list = [sentences[index] for index in top_sentences]
        # Merge the top 5 sentences into a paragraph
        paragraph = " ".join(top_sentences_list)
        print(paragraph)
        print("")



Top 5 answers for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
Document: rpojmv49
 Thus evolutionarily conserved proteins have been the best candidature for vaccine development  The SARS-Cov-2 emerged in Wuhan, China in Dec, 2019 and since then had spread to 213 countries  The study of its proteome organization could illustrate the proteins which act as the key molecular players in the infection cycle of the virus  Proteome analysis has also revealed the critical mutated proteins that are responsible for COVID-19 pathogenesis and virulence  Although DNA vaccine has also been attempted using RDT, but the high rate of mutation associated with COVID-19 have made such vaccines ineffective even before use

Document: wh9vvgv2
 The clinical features of patients infected by MERS-CoV are very similar to SARS, i  Thus, it is valuable to investigate the features of T-cell immunity against MERS-CoV in survivors, which may shed light on MERS-CoV vaccine development wit

##Doc2Vec

In [19]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import numpy as np
documents = dic
# Preprocess documents and create TaggedDocument objects
tagged_documents = []
for doc_id, doc_text in documents.items():
    tagged_documents.append(TaggedDocument(words=word_tokenize(doc_text.lower()), tags=[doc_id]))

# Train the Doc2Vec model
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4, epochs=50)

# Get the document embeddings
document_embeddings = {}
for doc_id, doc_text in documents.items():
    document_embeddings[doc_id] = model.infer_vector(word_tokenize(doc_text.lower()))

# Calculate the cosine similarity between queries and documents
cosine_similarities = []
for query in queries:
    query_embedding = model.infer_vector(word_tokenize(query.lower()))
    similarities = {}
    for doc_id, doc_embedding in document_embeddings.items():
        similarities[doc_id] = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
    cosine_similarities.append(similarities)

# Get the top 10 document IDs for each query
num_top_documents = 10
final = []
for i, query in enumerate(queries):
    documentlst=[]
    print(f"\nTop {num_top_documents} document IDs for query '{query}':")
    top_documents = sorted(cosine_similarities[i], key=cosine_similarities[i].get, reverse=True)[:num_top_documents]
    for j, doc_id in enumerate(top_documents):
        similarity = cosine_similarities[i][doc_id]
        print(f"{j+1}. {doc_id} (similarity: {similarity:.3f})")
        documentlst.append(doc_id)
    final.append(documentlst) 



Top 10 document IDs for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
1. 293et921 (similarity: 0.588)
2. riiyc6mt (similarity: 0.583)
3. zughvh0f (similarity: 0.581)
4. b8bl5vq5 (similarity: 0.547)
5. 3dgd54xr (similarity: 0.542)
6. 95zdshem (similarity: 0.533)
7. zdtq289a (similarity: 0.529)
8. kylls52u (similarity: 0.520)
9. ad5of9uv (similarity: 0.519)
10. dtn5t8ka (similarity: 0.514)

Top 10 document IDs for query 'How do cytokine pathways link sleep and immunity to infection and COVID-19?':
1. wpdgqhx9 (similarity: 0.455)
2. on1nclkj (similarity: 0.449)
3. ds7bmp6l (similarity: 0.442)
4. 3exvu20c (similarity: 0.431)
5. 6i98teyl (similarity: 0.406)
6. v84rzhin (similarity: 0.383)
7. 9hqoq7tf (similarity: 0.382)
8. zcktrmyg (similarity: 0.379)
9. bg1863z9 (similarity: 0.370)
10. 8jcs1aps (similarity: 0.369)

Top 10 document IDs for query 'When is it safe to discharge a COVID-19 patient with pneumonia? ':
1. rchkyihf (similarity: 0.775)
2. fsgkufwr (simil

In [20]:
final_sentences=[]
for i in final:
  lst_sentences=[]
  for j in i:
    with open('/content/2020-10-22-new/'+ j + '.json') as f:
      data = json.load(f)
      text = ' '.join([c['text'] for c in data['contexts']])
      sentences = text.split(".")
      lst_sentences.append(sentences)
  final_sentences.append(lst_sentences)   


In [21]:
import json
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


# Preprocess documents and create TaggedDocument objects
documents = {}
for i, doc_ids in enumerate(final):
    for doc_id in doc_ids:
        with open('/content/2020-10-22-new/'+ doc_id + '.json') as f:
            data = json.load(f)
            text = ' '.join([c['text'] for c in data['contexts']])
            documents[doc_id] = text

tagged_documents = []
for doc_id, doc_text in documents.items():
    tagged_documents.append(TaggedDocument(words=word_tokenize(doc_text.lower()), tags=[doc_id]))

# Train the Doc2Vec model
model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, workers=4, epochs=50)

# Calculate the cosine similarity between queries and documents
cosine_similarities = []
for query in queries:
    query_embedding = model.infer_vector(word_tokenize(query.lower()))
    similarities = {}
    for doc_id, doc_text in documents.items():
        doc_embedding = model.infer_vector(word_tokenize(doc_text.lower()))
        similarities[doc_id] = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
    cosine_similarities.append(similarities)


# Get the top 5 sentences from the top documents for each query
# Loop through each query and its corresponding top documents
for i, (query, documents) in enumerate(zip(queries, final_sentences)):
    print(f"\nTop 10 answers for query '{query}':")
    # Loop through each document and its corresponding sentences
    for j, doc_id in enumerate(final[i]):
        print(f"Document: {doc_id}")
        # Get the document text
        with open('/content/2020-10-22-new/'+ doc_id + '.json') as f:
            data = json.load(f)
            text = ' '.join([c['text'] for c in data['contexts']])
        # Infer the document embedding using the trained Doc2Vec model
        doc_embedding = model.infer_vector(word_tokenize(text.lower()))
        # Calculate the cosine similarity between the query and the document
        query_embedding = model.infer_vector(word_tokenize(query.lower()))
        similarity = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
        # Get the top 5 sentences
        sentences = text.split(".")
        sentence_embeddings = [model.infer_vector(word_tokenize(sentence.lower())) for sentence in sentences]
        cosine_similarities = [np.dot(query_embedding, sentence_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(sentence_embedding)) for sentence_embedding in sentence_embeddings]
        top_sentences = np.argsort(cosine_similarities)[-10:][::-1]
        top_sentences_list = [sentences[index] for index in top_sentences]
        # Merge the top 5 sentences into a paragraph
        paragraph = " ".join(top_sentences_list)
        print(paragraph)
        print("")





Top 10 answers for query 'What features of SARS-CoV2 are targeted in vaccine development? ':
Document: 293et921
 We tested for interaction between sex and EVT  We compared baseline characteristics and outcomes between men and women  With ordinal logistic regression, we evaluated the association between EVT and 90-day functional outcome for men and women separately, adjusted for potential confounders  Functional independence (modified Rankin Scale score, 0-2) at 90 days was reached by 318 women (39%) and 364 men (39%) 001), were smoking less often (30% versus 44%; P<0 Background and Purpose- Previous studies have reported less favorable outcome and less effect of endovascular treatment (EVT) after ischemic stroke in women than in men 001) than men  Results- We included 1762 patients in the analyses, of whom 833 (47%) were women  Our aim was to study the influence of sex on outcome and on the effect of EVT for ischemic stroke in recent randomized trials on EVT  The primary outcome was 9